In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
import sys
if ".." not in sys.path:
    sys.path.append("..")
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# CHAPITRE 7 - Ecrire un module

## III. Tests unitaires

il existe différentes façon de tester un code. Les tests unitaires permettent de vérifier le bon fonctionnement d'un module ou d'une fonction et de s'assurer que le comportement ne change pas lors d'une mise à jour de la fonction.  

On peut par exemple se servir d'assertions. voici un exemple de fonction renvoyant la somme des éléments d'une liste.

In [4]:
def somme(L):
    s = 0
    for val in L:
        s = s + val
    return s

Et les assertions permettant de tester le code

In [5]:
assert somme([1,2,3]) == 6
assert somme([]) == 0
assert somme([0.1,0.1,0.1]) == 0.3

AssertionError: 

On voit ici que les deux premières donnent le résultat attendu mais pas la troisième. Il ne s'agit pas ici d'un bug de la fonction mais d'une erreur due au stockage approché de certains float en mémoire.

In [ ]:
assert somme([1,2,3]) == 6
assert somme([]) == 0
assert abs(somme([0.1,0.1,0.1]) - 0.3) <= 1e-10

Si la fonction passe les tests, rien ne s'affiche.

Lorsque l'on écrit un module, il n'est pas utile de tester ce module à chaque import. On peut donc placer les assertion en fin de module comme par exemple 
```python
if __name__ == '__main__':
    assert somme([1,2,3]) == 6
    assert somme([]) == 0
    assert abs(somme([0.1,0.1,0.1]) - 0.3) <= 1e-10
```
Ainsi lorsque l'on execute le module comme script principal, les tests sont effectués mais pas lorsque l'on importe le module.

### Exercice 1
1. Reprenez le module 1 et le module 2 et ajoutez y des tests sous forme d'assertions. Testez ces modules et faites des essais d'importation.

Une methode interressante pour pouvoir pratiquer ces tests est le module **doctest**. Celui ci permet d'introduire les tests dans la docstring ce qui donne en plus d'une possibilité de tester le module, des exemples d'utilisation de la fonction directement dans l'aide. Ce module lance la commande et compare (à la virgule près) la sortie de l'interpréteur et ce qui est proposé dans le docstring.
La commande **doctest.testmod()** permet de réaliser les tests sur le module on peut par exemple finir un module par 
```python
import doctest
if __name__ == '__main__':
    doctest.testmod()
```
L'option facultative **verbose = True** permet quand à elle d'avoir une sortie un peu plus détaillée lorsque les tests sont positifs.  

Les fonctions ne sont pas testées dans l'ordre où elles apparaissent dans le module.

In [6]:
# Exemple d'une fonction somme qui passe tous les tests.
def somme(L):
    """
    L est une liste d'entiers. La fonction somme renvoie
    un entier correspondant à la somme des éléments de L
    
        
    Cas normal, liste d'entiers
    >>> somme([1,2,3])
    6
    
    Cas d'une liste vide
    >>> somme([])
    0

    On peut tester une assertion, dans ce cas seule la première et la dernière ligne sont testées.
    On peut donc remplacer le milieu de la sortie par ce que l'on veut dont ...
    >>> somme([1.,2.,3.])
    Traceback (most recent call last):
        ...
    AssertionError: Uniquement des entiers
    """
    for val in L:
        assert type(val) == int, "Uniquement des entiers"
    s = 0
    for val in L:
        s += val
    return s


# Exemple d'un test raté sur somme2, les tests suivants sont tout de même exécutés
def somme2(L):
    """
    cas d'une liste vide
    >>> somme2([])
    0
        
    cas d'une liste de floats
    >>> somme2([0.1,0.1,0.1])
    0.3
    
    cas normal d'une liste d'entiers
    >>> somme2([1,2,3])
    6
    """
    s = 0
    for val in L:
        s += val
    return s
 
# cas d'une erreur lors de la saisie de la sortie dans le test (oubli d'un espace)
def couple(x,y):
    """
    cas d'une liste vide
    >>> couple([],[])
    Traceback (most recent call last):
        ...
    IndexError: list index out of range
        
    erreur dans la sortie du docstring
    >>> couple([1,2], [3,4])
    (1,3)
    """
    return x[0],y[0]

import doctest
doctest.testmod()

**********************************************************************
File "__main__", line 55, in __main__.couple
Failed example:
    couple([1,2], [3,4])
Expected:
    (1,3)
Got:
    (1, 3)
**********************************************************************
File "__main__", line 33, in __main__.somme2
Failed example:
    somme2([0.1,0.1,0.1])
Expected:
    0.3
Got:
    0.30000000000000004
**********************************************************************
2 items had failures:
   1 of   2 in __main__.couple
   1 of   3 in __main__.somme2
***Test Failed*** 2 failures.


TestResults(failed=2, attempted=8)

### Exercice 2
1. Ajoutez des tests aux fonctions précédentes
2. utilisez l'option **verbose = True** et étudiez la sortie obtenue

## IV. Test driven development

Le principe est d'écrire:
1. Ecrire des tests définissant ce que l'on souhaite faire ou ne pas faire
2. Vérifier que ces tests échouent (on vérifie que les tests fonctionnent bien)
3. Ecrire le minimum de code permettant de passer ces tests
4. Verifier que tous les tests passent avec succès
5. Reprendre le code pour l'alléger et le restructurer en vérifiant que le conde continue de passer tous les tests

On itère ensuite le processus en ajoutant des tests définissant de nouvelles fonctionalités.
<p><a href="https://commons.wikimedia.org/wiki/File:TDD_Global_Lifecycle.png#/media/File:TDD_Global_Lifecycle.png"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/TDD_Global_Lifecycle.png/1200px-TDD_Global_Lifecycle.png" alt="TDD Global Lifecycle.png"></a><br>By <a href="//commons.wikimedia.org/w/index.php?title=User:Xarawn&amp;amp;action=edit&amp;amp;redlink=1" class="new" title="User:Xarawn (page does not exist)">Xarawn</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="https://creativecommons.org/licenses/by-sa/4.0" title="Creative Commons Attribution-Share Alike 4.0">CC BY-SA 4.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=44782343">Link</a></p>

Les tests tels qu'ils sont mis à profit dans TDD permettent d'explorer et de préciser le besoin, puis de spécifier le comportement souhaité du logiciel en fonction de son utilisation, avant chaque étape de codage. Le logiciel ainsi produit est tout à la fois pensé pour répondre avec justesse au besoin et conçu pour le faire avec une complexité minimale. On obtient donc un logiciel mieux conçu, mieux testé et plus fiable, autrement dit de meilleure qualité. 

### Exercice 3
1. Développez un algorithme de tri en respectant le principe du test driven development
2. Modifiez alors votre code afin qu'il permette de trier à la fois des listes d'entiers, des listes de floats ou des listes de chaînes de cractères (un seul type dans la liste) -cela ajoute ces cas aux spécifications de votre fonction mais ne devrait pas modifier le code-
3. Modifiez encore une fois votre code (en respectant ls principes du TDD) afin de trier une liste de points sous la forme de couples de floats [(x0, y0), (x1, y1), ... , (xn, yn)] par ordonnée croissant et par abscisse croissante dans le cas d'ordonnées égales.